In [1]:
import pandas as pd
from nba_api.live.nba.endpoints import todaysgameodds


/Users/rohilverma/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ImportError: cannot import name 'todaysgameodds' from 'nba_api.live.nba.endpoints' (/Users/rohilverma/Library/Python/3.9/lib/python/site-packages/nba_api/live/nba/endpoints/__init__.py)